## Fast Agent Example

FastAgency is an open-source framework designed to accelerate the development and deployment of multi-agent AI workflows. <br>
It builds upon existing agentic frameworks like AutoGen and plans to support others like CrewAI.  <br>
FastAgency provides a unified programming interface, seamless integration with external APIs, and tools to transition from prototype to production efficiently.

In [1]:
import os
import logging
from pathlib import Path
from dotenv import load_dotenv
from typing import Annotated
from autogen import register_function
from fastagency import UI, FastAgency
from fastagency.ui.console import ConsoleUI
from autogen.agentchat import ConversableAgent
from fastagency.api.openapi.client import OpenAPI
from fastagency.runtime.autogen import AutoGenWorkflows
from fastagency.api.openapi.security import APIKeyQuery


# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
load_dotenv(Path("../../.env"))


# Define the language model configuration
llm_config = {
    "config_list": [
        {
            "model": "gpt-3.5-turbo",
            "api_key": os.getenv("OPENAI_API_KEY"),
        }
    ],
    "temperature": 0.7,
}

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/opt/homebrew/anaconda3/envs/fastagency/lib/python3.11/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <B9202094-7D52-318C-99CF-7034B0E9F28D> /opt/homebrew/anaconda3/envs/fastagency/lib/python3.11/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/lib

In [ ]:
# Create workflows instance
wf = AutoGenWorkflows()

# Define the function to calculate photosynthetic efficiency
def calculate_photosynthetic_efficiency(
    light_intensity: Annotated[float, "Intensity of light in μmol photons m⁻² s⁻¹"],
    absorption_coefficient: Annotated[float, "Fraction of light absorbed by the plant"],
) -> float:
    """
    Calculate the photosynthetic efficiency based on light intensity and absorption coefficient.
    """
    efficiency = light_intensity * absorption_coefficient * 0.01  # Simplified formula
    return efficiency

# Define and register the workflow
@wf.register(name="learning_session", description="A student learns from a teacher.")
def learning_workflow(
    wf: AutoGenWorkflows, ui: UI, initial_message: str, session_id: str
) -> str:
    # Create the student agent
    student_agent = ConversableAgent(
        name="Student_Agent",
        system_message=(
            "You are a curious student eager to learn new things."
            "\nYou can fetch GIFs related to topics using the Giphy API."
        ),
        llm_config=llm_config,
    )
    # Create the teacher agent
    teacher_agent = ConversableAgent(
        name="Teacher_Agent",
        system_message=(
            "You are a knowledgeable teacher."
            "\nYou can calculate photosynthetic efficiency using the function "
            "'calculate_photosynthetic_efficiency(light_intensity, absorption_coefficient)'."
        ),
        llm_config=llm_config,
    )

    # Register the function
    register_function(
        calculate_photosynthetic_efficiency,
        caller=teacher_agent,
        executor=teacher_agent,
        name="calculate_photosynthetic_efficiency",
        description="Calculates photosynthetic efficiency based on light intensity and absorption coefficient.",
    )

    # Load and set up the Giphy API
    giphy_api = OpenAPI.create(openapi_json='giphy_openai.json')
    giphy_api.set_security_params(
        APIKeyQuery.Parameters(value=os.getenv("GIPHY_API_KEY", "your-giphy-api-key"))
    )

    # Register the API functions with the student agent
    wf.register_api(
        api=giphy_api,
        callers=student_agent,
        executors=student_agent,
    )

    # Start the conversation
    chat_result = student_agent.initiate_chat(
        teacher_agent,
        message=initial_message,
        summary_method="reflection_with_llm",
        max_turns=5,
    )

    # Return the conversation summary
    return chat_result.summary

In [ ]:
# Set up the FastAgency application
app = FastAgency(wf=wf, ui=ConsoleUI())

# Function to run the learning session
def run_learning_session():
    initial_message = "Can you explain the concept of photosynthesis?"
    session_id = "session_1"

    # Start the UI
    ui = ConsoleUI()
    ui.start(
        app=app,
        import_string=__name__,
        name="learning_session",
        initial_message=initial_message,
    )


In [ ]:
run_learning_session()
